# 🕵️‍♂️ Credit Card Fraud Detection – PySpark Notebook
Process, flatten, clean, and visualize JSON-formatted credit card transaction data.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.appName("CreditCardFraudProcessing").getOrCreate()

In [ ]:
# Load the raw JSON file
df_raw = spark.read.json("path/to/data.json")
df_raw.printSchema()
df_raw.show(3, truncate=False)

In [ ]:
# Define schema for personal_detail
personal_schema = StructType([
    StructField("person_name", StringType()),
    StructField("gender", StringType()),
    StructField("address", StringType()),
    StructField("lat", StringType()),
    StructField("long", StringType()),
    StructField("city_pop", StringType()),
    StructField("job", StringType()),
    StructField("dob", StringType())
])

# Parse the personal_detail JSON string
df = df_raw.withColumn("personal_detail_json", from_json("personal_detail", personal_schema)).drop("personal_detail")

In [ ]:
# Define address schema
address_schema = StructType([
    StructField("street", StringType()),
    StructField("city", StringType()),
    StructField("state", StringType()),
    StructField("zip", StringType())
])

df = df.withColumn("address_json", from_json("personal_detail_json.address", address_schema))

In [ ]:
# Extract and flatten fields
df = df \
    .withColumn("first", split(col("personal_detail_json.person_name"), "[,@/]")[0]) \
    .withColumn("last", split(col("personal_detail_json.person_name"), "[,@/]")[1]) \
    .withColumn("gender", col("personal_detail_json.gender")) \
    .withColumn("dob", col("personal_detail_json.dob")) \
    .withColumn("street", col("address_json.street")) \
    .withColumn("city", col("address_json.city")) \
    .withColumn("state", col("address_json.state")) \
    .withColumn("zip", col("address_json.zip")) \
    .withColumn("lat", col("personal_detail_json.lat").cast("double")) \
    .withColumn("long", col("personal_detail_json.long").cast("double")) \
    .withColumn("city_pop", col("personal_detail_json.city_pop").cast("int")) \
    .withColumn("job", col("personal_detail_json.job")) \
    .drop("personal_detail_json", "address_json")

In [ ]:
# Convert epoch microseconds to timestamp
def convert_epoch_microseconds(colname):
    return from_utc_timestamp((col(colname) / 1000000).cast("timestamp"), "Asia/Kuala_Lumpur")

df = df \
    .withColumn("trans_date_trans_time", to_timestamp("trans_date_trans_time")) \
    .withColumn("merch_last_update_time", convert_epoch_microseconds("merch_last_update_time")) \
    .withColumn("merch_eff_time", convert_epoch_microseconds("merch_eff_time"))

In [ ]:
# Mask sensitive credit card number
df = df.withColumn("cc_num_masked", sha2(col("cc_num"), 256))

In [ ]:
df.select("first", "last", "gender", "amt", "category", "is_fraud").show(5)

In [ ]:
# Convert to Pandas for visualization
pdf = df.select("category", "amt", "is_fraud").toPandas()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
sns.countplot(data=pdf, x="category", hue="is_fraud")
plt.title("Fraud Cases by Category")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()